In [10]:
import pandas as pd
import altair as alt
import geopandas as gpd

In [11]:
# load data and preprocess
url = 'https://raw.githubusercontent.com/xh0417/IV_group63/main/europe_covid.csv'
data = pd.read_csv(url)
data = data[data.Country != 'Channel Islands']
data = data[data.Country != 'Gibraltar']
data = data[data.Country != 'Isle of Man']
data.replace(['UK'], ['United Kingdom'], inplace=True)
data.replace(['Czechia'], ['Czech Republic'], inplace=True)
data.replace(['Faeroe Islands'], ['Faroe Islands'], inplace=True)
data.replace(['Moldova'], ['Republic of Moldova'], inplace=True)
data.replace(['North Macedonia'], ['The former Yugoslav Republic of Macedonia'], inplace=True)

data['Death - Total'] = '0-100'
data.loc[data['Total Deaths'] > 100, 'Death - Total'] = '100-1000'
data.loc[data['Total Deaths'] > 1000, 'Death - Total'] = '1000-10000'
data.loc[data['Total Deaths'] > 10000, 'Death - Total'] = '10000-100000'
data.loc[data['Total Deaths'] > 100000, 'Death - Total'] = '100000-200000'
data.loc[data['Total Deaths'] > 200000, 'Death - Total'] = '>200000'

# load EU map data
eu_data = gpd.read_file('https://raw.githubusercontent.com/xh0417/IV_group63/main/europe.geojson')

In [12]:
chart = alt.Chart(eu_data, width=1040, height=640).mark_geoshape().encode(
    alt.Color('Death - Total:N', scale=alt.Scale(domain=['0-100', '100-1000', '1000-10000', '10000-100000','100000-200000','>200000'], range=['#FCEEC6', '#D9A98B', '#D49166', '#C96B35','#AE5B2C','#8E4922'])),
    tooltip = ['NAME', 'Active Cases:Q', 'Total Cases:Q','Total Deaths:Q','Total Recovered:Q'],
).project(
    'naturalEarth1'
).transform_lookup(
    lookup='NAME',
    from_=alt.LookupData(data, 'Country', ['Active Cases','Total Cases','Total Deaths','Total Recovered','Death - Total'])
)
chart.save('Death - Total.html')